In [3]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from scipy.special import expit
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score

%matplotlib inline

In [4]:
class NeuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # количество узлов в слоях сети
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # начальные матрицы весовых коэффициентов
        # весовые коэффициенты между узлом i и узлом j
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        #коэффициент обучения
        self.lr = learningrate
        
        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: expit(x)
        
    # тренировка сети
    def train(self, inputs_list, targets_list):
        # преобразование списка входных значений в двумерный массив
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # расчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # расчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        # расчитать входные сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # расчитать исходящие сигналы для выходного слоя
        final_output = self.activation_function(final_inputs)
        
        #ошибки выходного слоя
        output_errors = targets - final_output
        # ошибки скрытого слоя 
        hidden_error = np.dot(self.who.T, output_errors)
        
        # обновить весовые коэффициенты для связей скрытого и выходного слоев
        a = (output_errors * final_output * (1.0 - final_output))
        b = np.transpose(hidden_outputs)
        self.who += self.lr * np.dot(a, b)
        # обновить весовые коэффициенты для связей входного и скрытого слоев
        self.wih += self.lr * np.dot(
            (hidden_error * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs)
        )
            
    # опрос нейронной сети
    def query(self, input_list):
            # преобразовать список входных значений в двумерный массив
            inputs = np.array(input_list, ndmin=2).T
            
            # расчитать входящие сигналы для скрытого слоя
            hidden_inputs = np.dot(self.wih, inputs)
            # расчитать выходные сигналы скрытого слоя
            hidden_outputs = self.activation_function(hidden_inputs)
            
            # расчитать входящие сигналы для выходного слоя
            final_inputs = np.dot(self.who, hidden_outputs)
            # расчитать выходные сигналы выходного слоя
            final_output = self.activation_function(final_inputs)
            
            return final_output

In [14]:
def scale_input_data(data):
    scaled_data = data / 255 * 0.99 + 0.01
    return scaled_data

In [19]:
def scale_labels(labels):
    onodes = 10
    targets = np.zeros((len(labels), onodes)) + 0.01
    indexes = range(len(labels))
    for idx, label in zip(indexes, labels):
        targets[idx][int(label)] = 0.99
    return targets

In [27]:
def train_network(network, X_train, y_train):
    for inputs, targets in zip(X_train, y_train):
        network.train(list(inputs), list(targets))

In [28]:
def query_network(network, X_test):
    network_outputs = []
    for inputs in X_test:
        output = network.query(list(inputs))
        digit = np.argmax(output)
        network_outputs.append(digit)
    return np.array(network_outputs)

### Загрузка и подготовка данных

In [32]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [33]:
X_train, y_train = train_data.drop(labels='label', axis=1).to_numpy(), train_data['label'].to_numpy()
X_test = test_data.copy().to_numpy()

In [34]:
X_train_scaled = scale_input_data(X_train)
y_train_scaled = scale_labels(y_train)
X_test_scaled = scale_input_data(X_test)

### Обучение нейронной сети и сохранение результатов

In [36]:
final_network_params = [784, 300, 10, 0.1]
final_n = NeuralNetwork(*final_network_params)
epochs = 6

for epoch in range(epochs):
    train_network(final_n, X_train_scaled, y_train_scaled)
y_pred = query_network(final_n, X_test_scaled)

In [154]:
result_array = np.zeros((X_test.shape[0], 2), dtype='int32')

for idx, label in zip(range(28000), y_pred):
    result_array[idx][0] = idx + 1
    result_array[idx][1] = label
    
result_array[1]

array([2, 0])

In [155]:
df = pd.DataFrame(result_array, columns=['ImageId', 'Label'])

In [156]:
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [157]:
type(df['ImageId'][1])

numpy.int32

In [158]:
df.to_csv('mnist_case_result.csv', index=False)